# Data Collection Scripts

In [73]:
import ssl
import pandas as pd
from tabula.io import read_pdf
import matplotlib.pyplot as plt

## Kenyan Data Cleaning
Outpiting CSV files for Kenyan cleaned data as it was initially collected from [Kenya's Energy and Petroleum Regulation Authority (EPRA)](https://www.epra.go.ke/services/petroleum/petroleum-prices/).

In [25]:
kenya_df = pd.read_csv('kenya_consolidated.csv')
kenya_df.head()

,start_date,end_date,town,super_petrol,diesel,kerosene
0,15/07/2023,14/08/2023,Mombasa,191.62,176.63,166.43
1,15/07/2023,14/08/2023,Kilifi,192.39,177.40,167.19
2,15/07/2023,14/08/2023,Likoni Mainland,192.00,177.02,166.81
3,15/07/2023,14/08/2023,Kwale,192.00,177.02,166.81
4,15/07/2023,14/08/2023,Malindi,192.62,177.62,167.41


In [26]:
kenya_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3531 entries, 0 to 3530
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_date    3531 non-null   object 
 1   end_date      3531 non-null   object 
 2   town          3531 non-null   object 
 3   super_petrol  3531 non-null   float64
 4   diesel        3531 non-null   float64
 5   kerosene      3531 non-null   float64
dtypes: float64(3), object(3)
memory usage: 165.6+ KB


In [27]:
## See the unique start dates and end dates (They should be have same unique values)
start_dates = kenya_df.start_date.unique()
end_dates = kenya_df.end_date.unique()
print("Start Dates: ", start_dates,start_dates.shape, "\n End Dates: ", end_dates, end_dates.shape)

Start Dates:  ['15/07/2023' '15/06/2023' '15/04/2023' '15/01/2023' '15/12/2022'
 '15/11/2022' '15/10/2022' '15/09/2022' '15/08/2022' '15/07/2022'
 '15/06/2022' '15/05/2022' '15/04/2022' '15/03/2022' '15/02/2022'
 '15/01/2022' '15/11/2021'] (17,) 
 End Dates:  ['14/08/2023' '14/07/2023' '14/05/2023' '14/02/2023' '14/01/2023'
 '14/12/2022' '14/11/2022' '14/10/2022' '14/09/2022' '14/08/2022'
 '14/07/2022' '14/06/2022' '14/05/2022' '14/04/2022' '14/03/2022'
 '14/02/2022' '14/12/2021'] (17,)


In [41]:
# Separate data by start date and save data in different CSV files named by the date
for date in start_dates:
    df = kenya_df[kenya_df.start_date == date]
    fname = "kenya_"+ date[-4:] + "_" + date[3:5] + "_" + date[:2] + ".csv" #Format file name: kenya_year_month_day.csv
    df.to_csv(fname)

You can now access the Kenyan data files based on their dates (As available through EPRA), or you can access all the data available from EPRA through the file `kenya_consolidated.csv`.

## Tanzania Data Collection
Tanzanian data from the [Energy and Water Utilities Authority (EWURA)](https://www.ewura.go.tz/fuel-prices/) was only available in PDF format. Below we'll design scripts to seamlessly extract data from the PDF documents into CSV files.

If there's an issue with the Java Environment, install `temurin`. In Mac/Linux, this can be done with `brew install temurin`

In [79]:
links = ["https://www.ewura.go.tz/wp-content/uploads/2023/10/Cap-Prices-wef-4th-October-2023-English-03102023-1.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/09/Cap-Prices-for-Petroleum-Products-wef-6th-September-2023-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/08/Cap-Prices-for-Petroleum-Products-wef-2nd-August-2023-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/07/Cap-Prices-for-Petroleum-Products-wef-5th-July-2023-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/06/Cap-Prices-for-Petroleum-Products-wef-7th-June-2023-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/05/Cap-Prices-for-Petroleum-Products-wef-3rd-May-2023-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/04/Cap-Prices-for-Petroleum-Products-wef-5th-April-2023-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/02/Cap-Prices-for-Petroleum-Products-wef-1st-March-2023-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/02/Cap-Prices-for-Petroleum-Products-wef-1st-February-2023-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2023/01/Cap-Prices-Publication-wef-4-January-2023-English-1.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/12/Cap-Prices-for-Petroleum-Products-wef-07-December-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/11/Cap-Prices-for-Petroleum-Products-wef-02-November-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/10/Cap-Prices-for-Petroleum-Products-wef-5-October-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/09/Cap-Prices-for-Petroleum-Products-wef-7-Sept-2022-English-.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/08/Cap-Prices-for-Petroleum-Products-wef-03-August-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/07/Cap-Prices-for-Petroleum-Products-wef-6-July-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/05/Cap-Prices-on-Petroleum-Products-wef-1st-June-2022-English-1.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/05/Cap-Prices-for-Petroleum-Products-wef-4-May-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/04/Cap-Prices-for-Petroleum-Products-wef-6-April-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/03/Cap-Prices-for-Petroleum-Products-wef-2-March-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/02/Cap-prices-for-Petroleum-Products-wef-2nd-February-2022-English-.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2022/01/Cap-Prices-for-Petroleum-Products-wef-05-January-2022-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/12/Cap-Prices-for-Petroleum-Products-wef-1-December-2021-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/11/Cap-Prices-for-Petroleum-Products-wef-3rd-November-2021-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/10/Cap-Prices-for-Petroleum-Products-wef-6-October-2021-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/09/Cap-Prices-for-Petroleum-Products-wef-02-September-2021-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/08/Cap-Prices-for-Petroleum-Products-wef-04-August-2021-English-2.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/08/Cap-Prices-for-Petroleum-Products-wef-01-July-2021-English-.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/08/Cap-Prices-for-Petroleum-Products-wef-02-June-2021-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/05/Cap-Prices-for-Petroleum-Products-wef-05-May-2021-English-.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/04/Cap-Prices-wef-07-April-2021-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/03/Cap-Prices-for-Petroleum-Products-wef-03-March-2021-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/08/Cap-Prices-for-Petroleum-Products-wef-03-February-2021-English.pdf",
"https://www.ewura.go.tz/wp-content/uploads/2021/01/Cap-Prices-for-Petroleum-Products-wef-06-January-2021-English.pdf"]

In [80]:
ssl._create_default_https_context = ssl._create_unverified_context #Fix SSL Certification Errors
for link in links:
    print(link)
    dfs = read_pdf(link, pages='all')
    for df in dfs:
        print("This is a new table:\n",df)

https://www.ewura.go.tz/wp-content/uploads/2023/10/Cap-Prices-wef-4th-October-2023-English-03102023-1.pdf
This is a new table:
            Ports Petrol (TZS/L) Diesel (TZS/L) Kerosene (TZS/L)
0  Dar es Salaam          3,281          3,448            2,943
1          Tanga          3,327          3,494            2,989
2         Mtwara          3,353          3,520            3,016
This is a new table:
    Unnamed: 0             Unnamed: 1 Unnamed: 2 Cap Prices Unnamed: 3
0        S.No                   Town        NaN        NaN        NaN
1         NaN                    NaN     Petrol     Diesel   Kerosene
2           1          Dar es Salaam      3,281      3,448      2,943
3           2                 Arusha      3,365      3,532      3,027
4           3    Arumeru (Usa River)      3,365      3,532      3,027
5           4                 Karatu      3,383      3,550      3,046
6           5                Longido      3,376      3,543      3,038
7           6                Mondu

Oct 09, 2023 3:52:41 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Oct 09, 2023 3:52:41 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Oct 09, 2023 3:52:46 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>


This is a new table:
           Region Petrol     Diesel Kerosen
0            NaN    NaN  TZS/Litre     NaN
1  Dar es Salaam  3,199      2,935   2,668
2          Tanga  3,245      2,981   2,740
3         Mtwara  3,271      3,008   2,714
This is a new table:
                          Town Cap Prices Unnamed: 0 Unnamed: 1
0                         NaN     Petrol     Diesel   Kerosene
1               Dar es Salaam      3,199      2,935      2,668
2                      Arusha      3,283      3,038      2,752
3         Arumeru (Usa River)      3,283      3,038      2,752
4                      Karatu      3,301      3,056      2,770
5                     Longido      3,294      3,049      2,763
6                     Monduli      3,289      3,043      2,757
7            Monduli-Makuyuni      3,294      3,048      2,762
8       Ngorongoro (Loliondo)      3,375      3,123      2,843
9              Coast (Kibaha)      3,204      2,940      2,672
10                   Bagamoyo      3,210      2,

Oct 09, 2023 3:58:03 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Oct 09, 2023 3:58:04 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Oct 09, 2023 3:58:04 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Oct 09, 2023 3:58:04 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Oct 09, 2023 3:58:04 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Oct 09, 2023 3:58:04 PM org.apache.pdfbox.contentstream.PDFStreamEngine operatorException
SEVERE: Cannot read JBIG2 image: jbig2-imageio is not installed
Oct 09, 2023 3:58:04 PM org.apache.pdfbox.contentstream.PDFStreamEngine oper

https://www.ewura.go.tz/wp-content/uploads/2021/08/Cap-Prices-for-Petroleum-Products-wef-04-August-2021-English-2.pdf
This is a new table:
                          Town           Cap Prices           Unnamed: 0  \
0                         NaN  Petrol\r(TZS/Litre)  Diesel\r(TZS/Litre)   
1               Dar es Salaam                2,427                2,251   
2                      Arusha                2,543                2,380   
3         Arumeru (Usa River)                2,543                2,380   
4                      Karatu                2,561                2,399   
5                     Longido                2,554                2,391   
6                     Monduli                2,548                2,386   
7            Monduli-Makuyuni                2,553                2,390   
8       Ngorongoro (Loliondo)                2,627                2,465   
9              Coast (Kibaha)                2,431                2,256   
10                   Bagamoyo      